# Loading&Preprocessing data


## Libraries

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import logging
from logging.handlers import RotatingFileHandler
import openpyxl

## Image Processing - raw data from videos



In [ ]:
# image processing


def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def convert_to_binary(frame, threshold=60):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray[:125, :] = 0  # Ignore the top 40 pixels
    _, binary = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
    return binary

def merge_close_contours(contours, min_distance=70, max_area_ratio_diff=0.7):
    merged_contours = []
    used = [False] * len(contours)
    
    def get_centroid(contour):
        M = cv2.moments(contour)
        if M['m00'] == 0:
            return None
        return (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
    
    for i, c1 in enumerate(contours):
        if used[i]:
            continue
        merged = [c1]
        centroid1 = get_centroid(c1)
        if centroid1 is None:
            continue
        area1 = cv2.contourArea(c1)
        for j, c2 in enumerate(contours):
            if i != j and not used[j]:
                centroid2 = get_centroid(c2)
                if centroid2 is None:
                    continue
                dist = np.linalg.norm(np.array(centroid1) - np.array(centroid2))
                area2 = cv2.contourArea(c2)
                area_ratio_diff = abs(area1 - area2) / max(area1, area2)
                
                # Merge only if the contours are close and have a similar area
                if dist < min_distance and area_ratio_diff < max_area_ratio_diff:
                    merged.append(c2)
                    used[j] = True
        merged_contours.append(np.vstack(merged))
        used[i] = True
    return merged_contours


def extract_contours(binary_frame):
    contours, _ = cv2.findContours(binary_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = merge_close_contours(contours)
    return contours

def frame_objects(contours):
    objects = []
    for contour in contours:
        if cv2.contourArea(contour) > 10:
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)
            box = np.int_(box)  # Recommended NumPy equivalent
            center, size, angle = rect
            if angle == 0:
                angle = 90
            rect = (center, size, angle)
            objects.append((rect, box, contour))
    return objects


def calculate_changes(new_object, old_object): # threshold calculations 
    changes = {}
    changes['centerX'] = np.linalg.norm(np.array(new_object[0][0][0]) - np.array(old_object[0][0][0]))
    changes['centerY'] = np.linalg.norm(np.array(new_object[0][0][1]) - np.array(old_object[0][0][1]))
    changes['width'] = abs(new_object[0][1][0] - old_object[0][1][0])
    changes['height'] = abs(new_object[0][1][1] - old_object[0][1][1])
    changes['rect_area'] = abs((new_object[0][1][0] * new_object[0][1][1]) - (old_object[0][1][0] * old_object[0][1][1]))
    changes['contour_area'] = abs(cv2.contourArea(cv2.convexHull(new_object[2])) - cv2.contourArea(cv2.convexHull(old_object[2])))
    return changes

def match_objects(objects, previous_objects, thresholds):
    matched_objects = {}
    for obj_id, (rect, _, contour) in previous_objects.items():
        best_match = None
        best_match_score = float('inf')
        for i, (current_rect, _, current_contour) in enumerate(objects):
            if i in matched_objects.values():
                continue
            current_object = (current_rect, current_rect[1], current_contour)
            changes = calculate_changes(current_object, (rect, rect[1], contour))
            match_score = 0
            for key in changes:
                if changes[key] > thresholds[key]:
                    match_score += 1
            if match_score < best_match_score:
                best_match_score = match_score
                best_match = i
        if best_match is not None and best_match_score == 0:  # All changes are within thresholds
            matched_objects[obj_id] = best_match
    return matched_objects

def save_object_parameters(objects, frame_idx, object_tracker):
    features = []
    for object_id, (rect, box, _) in object_tracker.items():
        center, size, angle = rect
        width, height = size
        area = width * height
        contour_area = cv2.contourArea(cv2.convexHull(box))
        area_ratio = contour_area / area if area != 0 else 0

        features.append({
            'frame': frame_idx,
            'object_id': object_id,
            'center_x': center[0],
            'center_y': center[1],
            'width': width,
            'height': height,
            'angle' : angle,
            'rect_area': area,
            'contour_area': contour_area,
            'area_ratio': area_ratio
        })
    return features

def process_video(video_path, output_csv, thresholds):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
      print(f"Error: Unable to open video {video_path}")
      return
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    all_features = []
    object_tracker = {}
    next_object_id = 1
    history = {}

    with tqdm(total=total_frames, desc="Processing video frames") as pbar:
        for frame_idx in range(total_frames):
            ret, frame = cap.read()
            if not ret:
                print(f"Error: Unable to read frame {frame_idx} in video {video_path}")
                break
            try:
                binary_frame = convert_to_binary(frame, thresholds['binary_threshold'])
                contours = extract_contours(binary_frame)
                objects = frame_objects(contours)
            except Exception as e:
                print(f"Error during processing frame {frame_idx}: {e}")
                continue         
               
            if frame_idx > 0:
                matched_objects = match_objects(objects, object_tracker, thresholds)
                new_object_tracker = {}
                for obj_id, i in matched_objects.items():
                    new_object_tracker[obj_id] = objects[i]
                    center, size, _ = objects[i][0]
                    history[obj_id] = (center, size)
                for i, (rect, box, contour) in enumerate(objects):
                    if i not in matched_objects.values():
                        center, size, _ = rect
                        best_match = None
                        for hist_id, (hist_center, hist_size) in history.items():
                            if np.linalg.norm(np.array(center) - np.array(hist_center)) < thresholds['centerX']:
                                if abs((size[0] * size[1]) - (hist_size[0] * hist_size[1])) / (size[0] * size[1]) < thresholds['rect_area']:
                                    best_match = hist_id
                                    break
                        if best_match is not None:
                            new_object_tracker[best_match] = (rect, box, contour)
                        else:
                            new_object_tracker[next_object_id] = (rect, box, contour)
                            history[next_object_id] = (center, size)
                            next_object_id += 1
                object_tracker = new_object_tracker
            else:
                for i, (rect, box, contour) in enumerate(objects):
                    object_tracker[next_object_id] = (rect, box, contour)
                    center, size, _ = rect
                    history[next_object_id] = (center, size)
                    next_object_id += 1

            frame_features = save_object_parameters(objects, frame_idx, object_tracker)
            all_features.extend(frame_features)

            for _, box, _ in objects:
                cv2.drawContours(frame, [box], 0, (0, 255, 0), 2)
            
            cv2.imshow("Processed Video", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
            pbar.update(1)

    cap.release()
    cv2.destroyAllWindows()

    df = pd.DataFrame(all_features)
    df.sort_values(by=['object_id', 'frame'], inplace=True)
    output_excel = output_csv.replace('.csv', '.xlsx')
    df.to_excel(output_excel, index=False, engine='openpyxl')


def process_all_videos(video_folder, output_folder, thresholds):
    ensure_dir(output_folder)
    for file in os.listdir(video_folder):
        if file.endswith('.mp4'):
            video_path = os.path.join(video_folder, file)
            output_csv = os.path.join(output_folder, f"{os.path.splitext(file)[0]}_features.csv")
            process_video(video_path, output_csv, thresholds)



In [ ]:
# Main image processing code
video_folder = r"C:\Users\thaim\OneDrive\Desktop\דוחות\test for flame ai\videos for train\fire\regular fire or fire with interference"
output_folder = r"c:\Users\thaim\OneDrive\Desktop\דוחות\test for flame ai\videos for train after code\fire\regular fire or fire with interference\test2"
# thresholds = {
#     'binary_threshold': 60,
#     'centerX': 10,
#     'centerY': 10,
#     'width': 5,
#     'height': 5,
#     'rect_area': 50,
#     'contour_area': 50
# }
thresholds = {
    'binary_threshold': 60,
    'centerX': 20,  # 
    'centerY': 20,
    'width': 10,  # 
    'height': 10,
    'rect_area': 100,  #
    'contour_area': 100
}



process_all_videos(video_folder, output_folder, thresholds)

## Image Preprocessing - Erase unwanted data


In [ ]:
import pandas as pd
import os

# 📌 הגדרת התיקייה שבה נמצאים קובצי ה-CSV שלך
input_folder = r"C:\Users\thaim\OneDrive\Desktop\דוחות\test for flame ai\videos for train after code\fire\regular fire or fire with interference\test_low_thresh"  # 🔄 שנה לתיקייה שלך
output_folder = os.path.join(input_folder, "clean data")
param_output_folder = os.path.join(output_folder, "parameters")

# 📂 יצירת תיקיות פלט אם הן לא קיימות
os.makedirs(output_folder, exist_ok=True)
os.makedirs(param_output_folder, exist_ok=True)

# 🔍 קבלת רשימת כל קובצי ה-CSV בתיקייה
csv_files = [f for f in os.listdir(input_folder) if f.endswith(".csv")]

# 🔥 סף להפרדה (ניתן לשנות בהתאם לצורך)
THRESHOLD_X = 50  # שינוי משמעותי ב-CENTER X
THRESHOLD_Y = 50  # שינוי משמעותי ב-CENTER Y

# 🚀 עיבוד כל קובץ בתיקייה
for file in csv_files:
    file_path = os.path.join(input_folder, file)
    df = pd.read_csv(file_path)

    # 1️⃣ מחיקת Bounding Boxes לא תקינים
    df_cleaned = df[
        (df['width'] >= 5) & 
        (df['height'] >= 5) & 
        (df['area_ratio'].between(0.8, 1.2)) & 
        (df['rect_area'] >= 10) & 
        (df['contour_area'] >= 10)
    ]

    # 2️⃣ מחיקת Bounding Boxes קטנים ליד Bounding Box גדול בכל פריים
    def filter_small_boxes(group):
        max_area = group['rect_area'].max()
        return group[group['rect_area'] >= 0.1 * max_area]

    df_cleaned = df_cleaned.groupby('frame', group_keys=False).apply(filter_small_boxes)

    # 3️⃣ מחיקת אובייקטים שהופיעו בפחות מ-30 פריימים
    object_counts = df_cleaned['object_id'].value_counts()
    valid_objects = object_counts[object_counts >= 30].index
    df_cleaned = df_cleaned[df_cleaned['object_id'].isin(valid_objects)]

    # 4️⃣ מחיקת שורות שבהן `center_x` ו-`center_y` לא משתנים בין פריימים (זיהוי הפרעות)
    df_cleaned['prev_center_x'] = df_cleaned.groupby('object_id')['center_x'].shift(1)
    df_cleaned['prev_center_y'] = df_cleaned.groupby('object_id')['center_y'].shift(1)

    df_cleaned = df_cleaned[
        (df_cleaned['center_x'] != df_cleaned['prev_center_x']) | 
        (df_cleaned['center_y'] != df_cleaned['prev_center_y'])
    ]

    # הסרת העמודות הזמניות
    df_cleaned.drop(columns=['prev_center_x', 'prev_center_y'], inplace=True)

    # **מיון לפי OBJECT ID תחילה, ולאחר מכן לפי FRAME**
    df_cleaned.sort_values(by=['object_id', 'frame'], inplace=True)

    # 5️⃣ הוספת שורות ריקות כאשר יש קפיצה גדולה במיקום הלהבה
    new_rows = []
    prev_row = None

    for _, row in df_cleaned.iterrows():
        if prev_row is not None:
            diff_x = abs(row['center_x'] - prev_row['center_x'])
            diff_y = abs(row['center_y'] - prev_row['center_y'])

            if diff_x > THRESHOLD_X or diff_y > THRESHOLD_Y:
                # הוספת שתי שורות ריקות
                new_rows.append(pd.Series([None] * len(row), index=df_cleaned.columns))
                new_rows.append(pd.Series([None] * len(row), index=df_cleaned.columns))

        new_rows.append(row)
        prev_row = row

    df_final = pd.DataFrame(new_rows, columns=df_cleaned.columns)

    # 6️⃣ שמירת הנתונים הנקיים לקובץ חדש בתיקייה "clean data"
    cleaned_file_path = os.path.join(output_folder, f"cleaned_{file}")
    df_final.to_csv(cleaned_file_path, index=False)

    print(f"✅ הקובץ '{file}' נשמר לאחר ניקוי בתיקייה: {cleaned_file_path}")

    # 7️⃣ **פיצול הנתונים לכל פרמטר לאחר שהקובץ עבר ניקוי**
    for object_id in df_final['object_id'].dropna().unique():
        df_obj = df_final[df_final['object_id'] == object_id]

        for column in ['center_x', 'center_y', 'width', 'height', 'angle', 'rect_area', 'contour_area', 'area_ratio']:
            df_param = df_obj[['frame', column]].copy()
            param_file_path = os.path.join(param_output_folder, f"{file.replace('.csv', '')}_object_{int(object_id)}_{column}.csv")
            df_param.to_csv(param_file_path, index=False)

    print(f"📂 כל הפרמטרים עבור הקובץ '{file}' נשמרו בתיקייה '{param_output_folder}'.")

print("🎉 כל הקבצים בתיקייה עברו ניקוי, פוצלו ונשמרו בתיקיית 'clean data'!")  


## Vector Functions - normalized and data preparation

In [ ]:

# def normalize_data(df):
#     for column in df.columns:
#         if column == "area_ratio":
#             continue
#         mean = df[column].mean()
#         std = df[column].std()
#         if std != 0:
#             df[column] = (df[column] - mean) / std
#         else:
#             df[column] = 0

#     return df

def normalize_data(df):
    for column in df.columns:  # Include all columns for normalization
        if column == "area_ratio":  # Skip normalization for 'area_ratio'
            continue    
        min_value = df[column].min()
        max_value = df[column].max()
        if max_value != min_value:  # Avoid division by zero
            df[column] = (df[column] - min_value) / (max_value - min_value)
        else:
            df[column] = 0  # If min and max are the same, set all entries to zero
    return df

def process_file(filepath, output_dir):
    filename = os.path.basename(filepath)
    df = pd.read_csv(filepath)

    if "fire" not in filename.lower():
        object_ids = df['object_id'].unique()
        for object_id in object_ids:
            object_df = df[df['object_id'] == object_id]
            process_and_save(object_df, filename, output_dir, object_id, is_fire=False)
    else:
        process_and_save(df, filename, output_dir, is_fire=True)

def process_and_save(df, filename, output_dir, object_id=None, is_fire=False):
    window_size = 20
    step_size = 5 
    folder = "fire" if is_fire else "interrupt"
    folder_path = os.path.join(output_dir, folder)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    for start in range(0, len(df) - window_size + 1, step_size):
        window_df = df.iloc[start:start+window_size].copy()
        if len(window_df) < window_size:
            continue

        window_df = window_df.drop(columns=["object_id", "frame"])
        window_df = window_df.reset_index(drop=True)
        window_df = normalize_data(window_df)

        output_filename = f"{filename.split('.')[0]}_object{object_id if object_id is not None else ''}_window{start}.csv"
        output_filepath = os.path.join(folder_path, output_filename)
        window_df.to_csv(output_filepath, index=False)

def main(input_dir, output_dir):

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".csv"):
                process_file(os.path.join(root, file), output_dir)

## Labeled Data CSV Functions - for specific model


In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import re

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text for text in re.split(r'(\d+)', s)]

def load_data_from_folder(folder_path):
    data_frames = []
    parameters = {
        'area_ratio': 1,
        'contour_area': 2,
        'rect_area': 3,
        'angle': 4,
        'height': 5,
        'width': 6,
        'centerX': 7,
        'centerY': 8
    }
    
    for parameter, label in parameters.items():
        print(f"Loading files for parameter: {parameter}")
        
        # Initialize tqdm progress bar
        parameter_files = [filename for filename in os.listdir(folder_path) if parameter in filename]
        parameter_files.sort(key=natural_sort_key)
        
        for filename in tqdm(parameter_files, desc=f"Processing {parameter} files"):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path, header=None)
            
            # Ensure we have 20 data points per file
            if len(df) != 20:
                raise ValueError(f"File {filename} does not have 20 data points.")
            
            df = df.T  # Transpose the DataFrame
            df.columns = [f'value_{i}' for i in range(1, 21)]
            df['label'] = label
            data_frames.append(df)

    combined_df = pd.concat(data_frames, ignore_index=True)
    return combined_df

def prepare_data(folder_path):
    # Load data
    df = load_data_from_folder(folder_path)
    
    return df


# Main code


## Creating Full CSV From Videos - calling Image processing


In [ ]:

thresholds = {
    'centerX': 120,          # Adjust as needed
    'centerY': 120,          # Adjust as needed
    'width': 100,            # Adjust as needed
    'height': 100,           # Adjust as needed
    'rect_area': 400,       # Adjust as needed
    'contour_area': 300     # Adjust as needed
    }

video_folder = r"C:\Users\thaim\OneDrive\Desktop\before done - videos for ai\fire"
output_folder = r"C:\Users\thaim\OneDrive\Desktop\before done - videos for ai\fire\data"
process_all_videos(video_folder, output_folder, thresholds)

## Creating Vectors For Model

In [ ]:
# creating vectors for models
input_directory = r"C:\Users\thaim\OneDrive\Desktop\before done - videos for ai\fire"
output_directory = r"C:\Users\thaim\OneDrive\Desktop\before done - videos for ai\vectors data"
main(input_directory, output_directory)

## Creating Prepared Data File With Labels - for specific model

In [ ]:

# folders = [
#     r"C:\Users\thaim\OneDrive\Desktop\test for flame ai\videos for train after code\fire_1_features",
#     r"C:\Users\thaim\OneDrive\Desktop\test for flame ai\videos for train after code\fire_2_features",
#     r"C:\Users\thaim\OneDrive\Desktop\test for flame ai\videos for train after code\fire_3_features",
#     r"C:\Users\thaim\OneDrive\Desktop\test for flame ai\videos for train after code\fire_4_features",
#     r"C:\Users\thaim\OneDrive\Desktop\test for flame ai\videos for train after code\fire_5_features",
#     r"C:\Users\thaim\OneDrive\Desktop\test for flame ai\videos for train after code\fire_6_features",   
# ]

# # Output directory for saving CSV files
# output_dir = r"C:\Users\thaim\OneDrive\Desktop\test for flame ai\videos for train after code\Interrupt_Objects"

# # Ensure the output directory exists
# os.makedirs(output_dir, exist_ok=True)

# # Loop through each folder and prepare data
# for folder_path in folders:
#     # Prepare data
#     prepared_df = prepare_data(folder_path)
    
#     # Extract folder name to use in the output file name
#     folder_name = os.path.basename(folder_path)
    
#     # Define the output file path
#     output_file = os.path.join(output_dir, f"{folder_name}_prepared_data.csv")
    
#     # Save prepared data to CSV
#     prepared_df.to_csv(output_file, index=False)
    
#     # Print a message indicating completion
#     print(f"Saved prepared data to {output_file}")


# Model codes



## train model

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Model
from keras.layers import Dense, Input, Concatenate, Dropout, Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

def load_data(input_dir):
    data = []
    labels = []
    processed_files = 0
    skipped_files = 0
    expected_shape = (20, 8)

    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".csv"):
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)

                if df.shape != expected_shape:  # Validate the shape of the dataframe
                    skipped_files += 1
                    continue

                if "fire" in file.lower():
                    label = 1  # Fire label
                else:
                    label = 0  # Disturbance label

                # Flatten the dataframe and append to data
                flattened_data = df.values.flatten()
                if flattened_data.shape[0] == np.prod(expected_shape):
                    data.append(flattened_data)
                    labels.append(label)
                    processed_files += 1
                else:
                    skipped_files += 1
                
    return np.array(data), np.array(labels)

# Original model with improvements
def create_model(input_shape):
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dense(1, activation='sigmoid'))  # 2 classes: fire and disturbance
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Grouped model as alternative
def create_grouped_model(input_shape):
    inputTensor = Input(shape=input_shape)
    
    group_layers = []
    for i in range(8):
        group = Lambda(lambda x: x[:, i*20:(i+1)*20], output_shape=(20,))(inputTensor)
        group = Dense(10, activation="relu")(group)
        group = Dense(5, activation="relu")(group)
        group = Dense(1, activation="relu")(group)
        group_layers.append(group)
    
    outputTensor = Concatenate()(group_layers)
    outputTensor = Dense(1, activation='sigmoid')(outputTensor)
    
    model = Model(inputTensor, outputTensor)
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

def main(input_directory, epochs=50, batch_size=32):
    data, labels = load_data(input_directory)
    
    # Reshape data to (number of files, 20 frames * 8 parameters)
    num_files = data.shape[0]
    data = data.reshape(num_files, 20 * 8)

    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=42)

    # Print the shape of the training data
    print(f'X_train shape: {X_train.shape}')
    
    # Choose the model type: create_model or create_grouped_model
    input_shape = (20 * 8,)
    model = create_model(input_shape)
    # model = create_grouped_model(input_shape)
    
    model.summary()
    
    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # Train the model
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[early_stopping])

    # Evaluate the model
    val_loss, val_acc = model.evaluate(X_val, y_val, verbose=2)
    print(f'Validation Loss: {val_loss}, Validation Accuracy: {val_acc}')
    
    # Save the model weights
    model.save_weights(r"C:\Users\thaim\OneDrive\Desktop\before done - videos for ai\vectors data\model_weights.weights.h5")


In [ ]:
# run model
if __name__ == "__main__":
    input_directory = r"C:\Users\thaim\Downloads\aivideo_20240818_060506_286.mp4" 
    main(input_directory)


## testing model

### testing model - with objects differentiate


In [ ]:
# testing model 2 - ID separated
import os
import time
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from collections import deque

# Configurable parameters
THRESHOLD_BINARY = 150
IGNORE_TOP_PIXELS = 125
MIN_CONTOUR_AREA = 0.1
MIN_DISTANCE_BETWEEN_CONTOURS = 120
MAX_AREA_RATIO_DIFF = 0.7
BUFFER_MAXLEN = 20
DISAPPEARANCE_THRESHOLD = 10
BORDER_THICKNESS = 5
LEARNING_RATE = 0.001
INPUT_SHAPE_MULTIPLIER = 8
PREDICTION_THRESHOLD = 0.8



# Initialize a global unique ID counter
next_id = 0

def create_model(input_shape):
    model = Sequential()
    model.add(Flatten(input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # 2 classes: fire and disturbance
    optimizer = Adam(learning_rate=LEARNING_RATE)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

def convert_to_binary(frame, threshold=THRESHOLD_BINARY):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray[:IGNORE_TOP_PIXELS, :] = 0  # Ignore the top pixels
    _, binary = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)
    return binary

def merge_close_contours(contours, min_distance=MIN_DISTANCE_BETWEEN_CONTOURS, max_area_ratio_diff=MAX_AREA_RATIO_DIFF):
    merged_contours = []
    used = [False] * len(contours)

    def get_centroid(contour):
        M = cv2.moments(contour)
        if M['m00'] == 0:
            return None
        return (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))

    for i, c1 in enumerate(contours):
        if used[i]:
            continue
        merged = [c1]
        centroid1 = get_centroid(c1)
        if centroid1 is None:
            continue
        area1 = cv2.contourArea(c1)
        for j, c2 in enumerate(contours):
            if i != j and not used[j]:
                centroid2 = get_centroid(c2)
                if centroid2 is None:
                    continue
                dist = np.linalg.norm(np.array(centroid1) - np.array(centroid2))
                area2 = cv2.contourArea(c2)
                area_ratio_diff = abs(area1 - area2) / max(area1, area2)

                if dist < min_distance and area_ratio_diff < max_area_ratio_diff:
                    merged.append(c2)
                    used[j] = True
        merged_contours.append(np.vstack(merged))
        used[i] = True
    return merged_contours

def extract_contours(binary_frame):
    contours, _ = cv2.findContours(binary_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = merge_close_contours(contours)
    return contours

def frame_objects(contours, frame_shape):
    objects = []
    frame_height, frame_width = frame_shape

    for contour in contours:
        if cv2.contourArea(contour) > MIN_CONTOUR_AREA:
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            center, size, angle = rect
            if angle == 0:
                angle = 90

            if any(point[0] <= BORDER_THICKNESS or point[1] <= BORDER_THICKNESS or 
                   point[0] >= frame_width - BORDER_THICKNESS or 
                   point[1] >= frame_height - BORDER_THICKNESS for point in box):
                continue

            rect = (center, size, angle)
            objects.append((rect, box, contour))
    return objects

def save_object_parameters(objects):
    all_features = []
    for rect, box, contour in objects:
        center, size, angle = rect
        width, height = size
        area = width * height
        contour_area = cv2.contourArea(cv2.convexHull(box))
        area_ratio = contour_area / area if area != 0 else 0

        features = [
            center[0], center[1], width, height, angle,
            area, contour_area, area_ratio
        ]
        all_features.append(features)
    return all_features

def normalize_features(feature_buffer):
    feature_array = np.array(feature_buffer)
    normalized_features = feature_array.copy()

    # for i in range(8):
    #     if i == 7: 
    #         continue
    #     param_values = feature_array[:, i::8]
    #     param_mean = param_values.mean()
    #     param_std = param_values.std()
    #     if param_std != 0:
    #         normalized_features[:, i::8] = (param_values - param_mean) / param_std
    #     else:
    #         normalized_features[:, i::8] = 0

    # return normalized_features

    for i in range(8):
        if i == 7: 
            continue
        param_values = feature_array[:, i::8]
        param_min = param_values.min()
        param_max = param_values.max()
        if param_max != param_min:
            normalized_features[:, i::8] = (param_values - param_min) / (param_max - param_min)
        else:
            normalized_features[:, i::8] = 0

    return normalized_features

def get_object_key(center, object_buffers, threshold=MIN_DISTANCE_BETWEEN_CONTOURS):
    for key, buffer in object_buffers.items():
        existing_center = np.array(buffer[-1][:2])
        if np.linalg.norm(np.array(center) - existing_center) < threshold:
            return key
    return None

def process_frame(frame, model, frame_counter, object_buffers, last_seen_frame):
    global next_id  # Use the global next_id to assign new unique IDs
    binary_frame = convert_to_binary(frame)
#    cv2.imshow('binary frame', binary_frame)
    contours = extract_contours(binary_frame)
    objects = frame_objects(contours, frame.shape[:2])

    if len(objects) == 0:
        return None, []

    features = save_object_parameters(objects)

    # Append features to the respective object's buffer
    for obj_features in features:
        if len(obj_features) == 8:
            center = obj_features[:2]
            key = get_object_key(center, object_buffers)
            if key is None:
                key = next_id
                next_id += 1
                object_buffers[key] = deque(maxlen=BUFFER_MAXLEN)
            object_buffers[key].append(obj_features)
            last_seen_frame[key] = frame_counter

    # Make prediction for each object's buffer
    predictions = {}
    for key, buffer in object_buffers.items():
        if len(buffer) == BUFFER_MAXLEN:
            normalized_features = normalize_features(list(buffer))            
            normalized_features = np.transpose(normalized_features)
            normalized_features = normalized_features.flatten().reshape(1, -1)
            prediction = model.predict(normalized_features)
            predictions[key] = prediction[0][0]

    return predictions, objects

def draw_tracking_info(frame, object_buffers, predictions):
    for key in object_buffers.keys():
        buffer = object_buffers[key]
        if len(buffer) > 0:
            # Use the latest buffer data for drawing
            obj_features = buffer[-1]
            center_x, center_y, width, height, angle = obj_features[:5]
            area, contour_area, area_ratio = obj_features[5:8]

            # Calculate the box points from the rectangle parameters
            rect = ((center_x, center_y), (width, height), angle)
            box = cv2.boxPoints(rect)
            box = np.int0(box)

            # Determine the color based on prediction
            color = (0, 255, 0) if predictions.get(key, 0) < PREDICTION_THRESHOLD else (0, 0, 255)
            cv2.drawContours(frame, [box], 0, color, 2)
            cv2.putText(frame, f"ID: {key}", (int(center_x), int(center_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            cv2.putText(frame, f"Pred: {predictions.get(key, 0):.2f}", (int(center_x), int(center_y) + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

def main(model_path, video_source):
    input_shape = (BUFFER_MAXLEN * INPUT_SHAPE_MULTIPLIER,)
    model = create_model(input_shape)
    model.load_weights(model_path)
    time.sleep(0.1)
    cap = cv2.VideoCapture(video_source)
    if not cap.isOpened():
        print("Error: Could not open video source.")
        return

    frame_counter = 0
    object_buffers = {}  # Dictionary to store buffers for each object
    last_seen_frame = {}  # Dictionary to store the last frame an object was seen

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        time.sleep(0.1)
        frame_counter += 1
        predictions, _ = process_frame(frame, model, frame_counter, object_buffers, last_seen_frame)
        
        # Remove objects not seen for 50 frames
        keys_to_delete = [key for key, last_frame in last_seen_frame.items() if frame_counter - last_frame > DISAPPEARANCE_THRESHOLD]
        for key in keys_to_delete:
            print(f"Deleting object {key} from buffers and last seen frames.")
            del object_buffers[key]
            del last_seen_frame[key]

        if predictions:
            draw_tracking_info(frame, object_buffers, predictions)

        cv2.imshow('Real-time Fire Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()




In [ ]:



if __name__ == "__main__":
    model_path = r"C:\Users\thaim\OneDrive\Desktop\before done - videos for ai\vectors data\model_weights.weights.h5"
    video_path = r"C:\Users\thaim\Videos\Recording 2024-08-18 141852 - IR_LED_1-5hz_5m_crop.mp4"
    main(model_path, video_path)  # Use 0 for webcam, or provide a video file path